## Introduction

**Project description**: lorem ipsum: 

High level steps (table of content, internal links) goes here: 

On [this link (Kaggle)](https://www.kaggle.com/c/titanic/data) you can find the original dataset and its description.

*Project owner*: ***Gustaf Olofsson***

## Data dictionary
**PassengerId**: type should be integers  
**Survived**: Survived or Not  
**Pclass**: Class of Travel  
**Name**: Name of Passenger  
**Sex**: Gender  
**Age**: Age of Passengers  
**SibSp**: Number of Sibling/Spouse aboard  
**Parch**: Number of Parent/Child aboard  
**Ticket**:   
**Fare**  
**Cabin**  
**Embarked**: The port in which a passenger has embarked. C - Cherbourg, S - Southampton, Q = Queenstown  

## Load Data from CSVs

In [15]:
import pandas as pd
import numpy as np
import os

dir_path = os.path.dirname(os.path.realpath('__file__'))
raw_data = pd.read_csv(dir_path+"\\titanic_data.csv")

#test that data loaded correctly
print(raw_data.head())

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  


## SECTION TITLE

Initially I want to explore whether the following factors differ between the survivors and non-survivors: 
* Ticket class  
* Gender
* Age
* Port in which passenger embarked from

In [17]:
#Divide dataset into survivors and non-survivors
survived = raw_data[raw_data.Survived==1]
not_survived = raw_data[raw_data.Survived==0]

print(survived.head(3))
print(not_survived.head(3))

   PassengerId  Survived  Pclass  \
1            2         1       1   
2            3         1       3   
3            4         1       1   

                                                Name     Sex   Age  SibSp  \
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   

   Parch            Ticket     Fare Cabin Embarked  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
   PassengerId  Survived  Pclass                      Name   Sex   Age  SibSp  \
0            1         0       3   Braund, Mr. Owen Harris  male  22.0      1   
4            5         0       3  Allen, Mr. William Henry  male  35.0      0   
5            6         0       3          Moran, Mr. James  male   NaN      0   

   Parch 

In [ ]:
def describe_data(df, attribute):
    print("for the {}-dataframe, looking at {}".format(df, attribute))
    
    return 0

## SECTION TITLE

## SECTION TITLE

## SECTION TITLE

## SECTION TITLE